In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from MC import *


## Task 2: Blocking analysis

In this task, you will study the correlation present in the Monte Carlo simulations
from task 1.
As you saw in the last exercise, block averaging can be used to obtain more
reliable error estimates when sampling is correlated, like in Metropolis sampling.

Continuing with the same system of `200` particles at Lennard-Jones temperature
and pressure of `T=2.0` and `ρ=0.5` (remember the equlibration!), plot the
evolution of block-averaged errors of the energy and pressure w.r.t. the blocking
iteration. Increase the number of transforms until the formation of a plateau
can be observed. At the onset of the plateau, the block size (how much of the
original data that has been reduced to one point) becomes longer than the
correlation length. Note that the sampling frequency `fsamp` in the Monte Carlo
simulation will affect the interpretation of the blocking analysis!

Compare the magnitude of errors computed with and without block-averaging, and
compare the correlation length with the number of equilibration steps.

In [ ]:
def block_transform(x: np.ndarray) -> np.ndarray:
    # Drop the last data point if the number of points is odd
    n = 2 * (x.shape[0] // 2)
    # Average adjacent values
    return (x[:n:2] + x[1:n:2]) / 2

def blocking_analysis(x: np.ndarray, n_transformations: int):
    if 2**n_transformations > data.shape[0]:
        raise Exception(
            f'Data allows for a maximum of {int(np.log(x.shape[0]) / np.log(2))} transformations.'
        )

    # Number of samples: N(M=0)
    N_0 = x.shape[0]
    # Variance: \sigma_A^2(M=0) = 1/N(M=0) * SUM[ (xi - mean(x))^2 ]
    var_0 = np.var(x)

    result = np.zeros((n_transformations + 1, 5))

    x_M = x.copy()
    for M in range(0, n_transformations + 1):
        N_M = x_M.shape[0]
        var_M = np.var(x_M)

        # Blocked correlation time \tau(M) = var_M 2**M / var_0 | Eq (26)
        correlation_time = var_M * 2**M / var_0
        # Standard error of the blocked correlation time = sqrt(2/N(M)) * \tau(M) | Eq (36)
        stddev_correlation_time = np.sqrt(2 / N_M) * correlation_time

        # Blocked error of the sum \sigma_I(M) = sqrt( var(M) 2**M / N_0 ) | Eq (27)
        stat_err = np.sqrt(var_M * 2**M / N_0)
        # Standard error of the blocked error of the sum = 1 / sqrt(2 N(M)) \sigma_I(M) | Eq (38)
        stddev_stat_err = 1 / np.sqrt(2 * N_M) * stat_err

        result[M,:] = [M, correlation_time, stddev_correlation_time, stat_err, stddev_stat_err]

        # Average neighboring elements
        x_M = block_transform(x_M)

    return result


In [ ]:
## Parameters
# MC parameters (used to define file to load)
rho = 0.5
temp = 2.0
quantity = 'pressure'  # 'pressure' or 'energy'
# Number of blocking transformations
# (< log_2(N) where N is the number of data points)
n_transformations = 4

## Load MC data
filename = f'sc_{quantity}_rho={rho:0.4f}_T={temp:0.4f}.dat'
data = np.loadtxt(filename)

## Blocking analysis
blocking_res_dict = {}
for n_data in [data.shape[0]]:
    blocking_res_dict[n_data] = blocking_analysis(data[:n_data], n_transformations)

## Plot
fig, axes = plt.subplots(1, 2, figsize=(8,4))
for (n_data, blocking_res) in blocking_res_dict.items():
    axes[0].errorbar(blocking_res[:,0], blocking_res[:,1], marker='.',
                    yerr=blocking_res[:,2], capsize=3, label=f'N={n_data}')
    axes[1].errorbar(blocking_res[:,0], blocking_res[:,3], marker='.',
                 yerr=blocking_res[:,4], capsize=3, label=f'N={n_data}')

axes[0].set_ylabel(r'$\tau(M)$')
axes[0].set_xlabel(r'$M$')
axes[0].set_xticks(blocking_res[::2,0])
axes[0].legend()

axes[1].set_ylabel(r'$\sigma_I(M)$')
axes[1].set_xlabel(r'$M$')
axes[1].set_xticks(blocking_res[::2,0])
axes[1].legend()

fig.tight_layout()
